# Step 2: Private Prediction using Syft Keras - Serving (Server)

In [1]:
from collections import OrderedDict
#import torch
import tensorflow as tf
import numpy as np
import syft
import tf_encrypted as tfe
# hook = syft.TorchHook(torch)
# alice = syft.VirtualWorker(hook, id="alice")  # alice computes on behalf of data owner
# bob = syft.VirtualWorker(hook, id="bob")      # bob computes on behalf of model owner
# carol = syft.VirtualWorker(hook, id="carol")  # carol acts as crypto producer

In [2]:
#import tf_encrypted as tfe
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

## Protocol

In [3]:
players = OrderedDict([
    ('server0', 'localhost:4000'),
    ('server1', 'localhost:4001'),
    ('server2', 'localhost:4002'),
])

config = tfe.RemoteConfig(players)
config.save('/tmp/tfe.config')

In [4]:
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

## Launching servers

Before actually serving the computation below we need to launch TensorFlow servers in new processes. Run the following in three different terminals. You may have to allow Python to accept incoming connections.

In [5]:
for player_name in players.keys():
    print("python -m tf_encrypted.player --config /tmp/tfe.config {}".format(player_name))

python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2


## Model

In [6]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [7]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_size=1, input_shape=task_shape[1:]))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# Load pre-trained weights
model.load_weights(pre_trained_weights)

In [9]:
data = np.ones([1, 28, 28, 1])

In [10]:
public_prediction = model.predict(data)
print("Public Prediction:", public_prediction)

Public Prediction: [[  6.3107085   -7.9984264    2.6922388   -0.4583856   -3.920539
    0.30345103   7.9021144  -15.255635     5.1295314   -7.754915  ]]


### Secure the model by sharing the weights

In [11]:
prot = tfe.protocol.SecureNN()

tfe_model = model.share(prot=prot)

INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


seq__init__ <tf_encrypted.protocol.securenn.securenn.SecureNN object at 0x1426fdf28>


InvalidArgumentError: Cannot assign a device for operation private-var/Variable: node private-var/Variable (defined at /Users/yanndupis/Documents/dropoutlabs/tfe_clean_copy/tf-encrypted/tf_encrypted/tensor/native.py:420) was explicitly assigned to /job:tfe/replica:0/task:0/device:CPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[node private-var/Variable (defined at /Users/yanndupis/Documents/dropoutlabs/tfe_clean_copy/tf-encrypted/tf_encrypted/tensor/native.py:420) ]]

Caused by op 'private-var/Variable', defined at:
  File "/anaconda3/envs/pysyft_36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/envs/pysyft_36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/envs/pysyft_36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anacon

## Computation

In [ ]:
input_shape = (1, 28, 28, 1)
output_shape = (1, 10)

def computation(x):
    return tfe_model(x)

In [ ]:
server = tfe.serving.QueueServer(
    input_shape=input_shape,
    output_shape=output_shape,
    computation_fn=computation)

In [ ]:
sess = tfe.Session(disable_optimizations=True)

tf.Session.reset(sess.target)

### Serving

In [ ]:
def step_fn():
    print("Next")

server.run(
    sess,
    num_steps=5,
    step_fn=step_fn)